In [22]:
import pandas as pd
import numpy as np
import scipy.stats as st
import seaborn as sns
import sys
import os
import gseapy as gp
import pdb
import copy
from scipy import sparse
import anndata
import cerberus
import plotly.io as pio

p = os.path.dirname(os.path.dirname(os.getcwd()))
sys.path.append(p)

from scripts.utils import *
from scripts.plotting import *

## 220830 more trying to figure out discordance b/w major and det obs cats

In [72]:
df = pd.read_csv('swan/isos_sample_gene_90.tsv', sep='\t')
df.loc[df.gid.str.contains('ENSG00000149932')].head()

,tid,gname,gid,sample,pi,pi_rank
454647,"ENSG00000149932[1,14,1]",TMEM219,ENSG00000149932.16,muscle,57.142860,1
485548,"ENSG00000149932[1,14,1]",TMEM219,ENSG00000149932.16,liver,52.173912,1
526452,"ENSG00000149932[1,14,1]",TMEM219,ENSG00000149932.16,brain,49.290650,1
546331,"ENSG00000149932[3,4,1]",TMEM219,ENSG00000149932.16,h9_panc_beta,46.153847,1
548545,"ENSG00000149932[1,14,1]",TMEM219,ENSG00000149932.16,heart,45.729538,1


In [65]:
df = pd.read_csv('swan/isos_90_in_progress.tsv', sep='\t')
max_total_pis = df[['gname_gid_biosamp', 'max_total_pi']]
df.drop('max_total_pi', axis=1, inplace=True)

In [66]:
df.loc[df.gid.str.contains('ENSG00000149932')].head()

,Unnamed: 0,tid,gname,gid,sample,pi,pi_rank,gname_gid_biosamp
454647,454647,"ENSG00000149932[1,14,1]",TMEM219,ENSG00000149932.16,muscle,57.142860,1,TMEM219_ENSG00000149932.16_muscle
485548,485548,"ENSG00000149932[1,14,1]",TMEM219,ENSG00000149932.16,liver,52.173912,1,TMEM219_ENSG00000149932.16_liver
526452,526452,"ENSG00000149932[1,14,1]",TMEM219,ENSG00000149932.16,brain,49.290650,1,TMEM219_ENSG00000149932.16_brain
546331,546331,"ENSG00000149932[3,4,1]",TMEM219,ENSG00000149932.16,h9_panc_beta,46.153847,1,TMEM219_ENSG00000149932.16_h9_panc_beta
548545,548545,"ENSG00000149932[1,14,1]",TMEM219,ENSG00000149932.16,heart,45.729538,1,TMEM219_ENSG00000149932.16_heart


In [67]:
pi_ranks = [1,2,3]
temp = df.loc[df.pi_rank.isin(pi_ranks)].groupby(['gname_gid_biosamp']).sum().reset_index()

In [68]:
temp.loc[temp.gname_gid_biosamp.str.contains('ENSG00000149932')]

,gname_gid_biosamp,Unnamed: 0,pi,pi_rank
551764,TMEM219_ENSG00000149932.16_a673,2231273,82.828281,6
551765,TMEM219_ENSG00000149932.16_adipose,2281848,78.333332,6
551766,TMEM219_ENSG00000149932.16_adrenal gland,2136428,90.796017,6
551767,TMEM219_ENSG00000149932.16_brain,2188106,93.815932,6
551768,TMEM219_ENSG00000149932.16_caco2,2129291,92.424246,6
551769,TMEM219_ENSG00000149932.16_calu3,2161890,89.237667,6
551770,TMEM219_ENSG00000149932.16_colon,2218667,83.284454,6
551771,TMEM219_ENSG00000149932.16_gm12878,2187496,85.714284,6
551772,TMEM219_ENSG00000149932.16_h1,2357850,79.761902,6
551773,TMEM219_ENSG00000149932.16_h1_de,2328474,75.966854,6


In [69]:
temp = temp.merge(max_total_pis, how='left', on='gname_gid_biosamp')
temp.loc[temp.max_total_pi >= 90, 'max_total_pi'] = 90
temp = temp.loc[temp.pi >= temp.max_total_pi]

In [70]:
temp.head()

,gname_gid_biosamp,Unnamed: 0,pi,pi_rank,max_total_pi
2,A1BG-AS1_ENSG00000268895.5_adipose,1191041,80.0,3,80.0
3,A1BG-AS1_ENSG00000268895.5_adipose,1191041,80.0,3,80.0
13,A1BG-AS1_ENSG00000268895.5_caco2,1465863,60.0,3,60.0
14,A1BG-AS1_ENSG00000268895.5_caco2,1465863,60.0,3,60.0
24,A1BG-AS1_ENSG00000268895.5_h1,36438,100.0,1,90.0


In [71]:
temp.loc[temp.gname_gid_biosamp.str.contains('ENSG00000149932')].head()

,gname_gid_biosamp,Unnamed: 0,pi,pi_rank,max_total_pi
2206566,TMEM219_ENSG00000149932.16_adrenal gland,2136428,90.796017,6,90.0
2206567,TMEM219_ENSG00000149932.16_adrenal gland,2136428,90.796017,6,90.0
2206568,TMEM219_ENSG00000149932.16_adrenal gland,2136428,90.796017,6,90.0
2206569,TMEM219_ENSG00000149932.16_adrenal gland,2136428,90.796017,6,90.0
2206570,TMEM219_ENSG00000149932.16_adrenal gland,2136428,90.796017,6,90.0


## idk

In [23]:
filt_ab = 'cerberus_filtered_abundance.tsv'
df = pd.read_csv(filt_ab, sep='\t')
df, tids = get_tpm_table(df,
               how='iso',
               min_tpm=1)

Calculating iso TPM values


/Users/fairliereese/Documents/programming/mortazavi_lab/data/rnawg/scripts/utils.py:1599: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[tpm_col] = (df[d]*1000000)/df[total_col]
/Users/fairliereese/Documents/programming/mortazavi_lab/data/rnawg/scripts/utils.py:1598: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[total_col] = df[d].sum()


Enforcing minimum TPM
Total # isos detected: 234040
# isos >= 1 tpm: 212911
Number of isos reported: 212911


In [24]:
df.loc[df.index.str.contains('ENSG00000149932')]

,mcf7_1_1,k562_1_1,heart_right_ventricle_2_1,gm12878_3_1,heart_right_ventricle_3_1,pc3_1_1,lower_lobe_of_right_lung_1_1,hl60_m1_24hr_1_1,hl60_m1_24hr_1_2,hl60_m2_24hr_1_1,...,heart_right_ventricle_1_1,right_lobe_of_liver_2_1,hl60_1_1,hl60_1_2,pgp1_endo_1_1,pgp1_endo_1_2,left_cardiac_atrium_1_1,h9_neural_crest_1_1,h9_neural_crest_1_2,left_lung_2_1
annot_transcript_id,,,,,,,,,,,,,,,,,,,,,
"ENSG00000149932[1,1,1]",20.606409,18.756891,18.584597,17.413848,16.811502,21.877817,38.671528,8.028888,17.128567,14.699277,...,7.376454,2.359064,9.749646,3.991540,53.431417,94.119025,14.681782,1.542722,3.457974,57.808531
"ENSG00000149932[1,10,1]",0.000000,1.631034,0.000000,3.265096,0.000000,2.625338,3.515593,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.723594,0.000000,0.000000,0.000000,0.000000,0.000000
"ENSG00000149932[1,11,1]",0.000000,0.815517,0.000000,0.000000,1.400959,0.000000,2.343729,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.498942,1.723594,0.000000,1.835223,0.000000,1.152658,0.000000
"ENSG00000149932[1,12,1]",1.873310,1.631034,0.000000,3.265096,0.000000,7.876014,1.171864,6.021666,1.557142,0.000000,...,0.922057,0.000000,2.543386,1.995770,1.723594,0.000000,0.000000,0.000000,0.000000,0.000000
"ENSG00000149932[1,13,1]",0.000000,0.000000,0.000000,5.441827,0.000000,0.875113,0.000000,0.000000,1.557142,1.633253,...,0.000000,0.000000,0.000000,0.498942,0.000000,0.000000,0.000000,0.000000,0.000000,1.751774
"ENSG00000149932[1,14,1]",24.353029,24.465510,41.815343,20.678944,37.825880,20.127591,63.280681,26.093886,15.571425,17.965783,...,7.376454,2.359064,16.532009,6.985195,70.667358,110.851296,9.176113,0.000000,3.457974,106.858194
"ENSG00000149932[1,15,1]",0.000000,0.815517,2.323075,0.000000,1.400959,0.000000,2.343729,0.000000,1.557142,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.835223,0.000000,0.000000,1.751774
"ENSG00000149932[1,16,1]",1.873310,0.815517,0.000000,1.088365,0.000000,5.250676,7.031187,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.847795,1.496827,0.000000,0.000000,1.835223,0.000000,1.152658,0.000000
"ENSG00000149932[1,3,1]",1.873310,1.631034,0.000000,1.088365,2.801917,0.875113,1.171864,0.000000,0.000000,1.633253,...,0.000000,0.000000,0.847795,1.496827,0.000000,0.000000,5.505668,0.000000,0.000000,0.000000


In [25]:
temp = df.loc[df.index.str.contains('ENSG00000149932')].transpose()

In [26]:
temp.head()

annot_transcript_id,"ENSG00000149932[1,1,1]","ENSG00000149932[1,10,1]","ENSG00000149932[1,11,1]","ENSG00000149932[1,12,1]","ENSG00000149932[1,13,1]","ENSG00000149932[1,14,1]","ENSG00000149932[1,15,1]","ENSG00000149932[1,16,1]","ENSG00000149932[1,3,1]","ENSG00000149932[1,3,2]","ENSG00000149932[1,4,1]","ENSG00000149932[1,9,5]","ENSG00000149932[2,2,1]","ENSG00000149932[3,10,1]","ENSG00000149932[3,16,1]","ENSG00000149932[3,4,1]","ENSG00000149932[5,10,1]"
mcf7_1_1,20.606409,0.000000,0.000000,1.873310,0.000000,24.353029,0.000000,1.873310,1.873310,1.873310,1.873310,0.0,0.0,0.000000,0.936655,26.226339,1.873310
k562_1_1,18.756891,1.631034,0.815517,1.631034,0.000000,24.465510,0.815517,0.815517,1.631034,0.000000,3.262068,0.0,0.0,4.077585,0.815517,23.649993,0.815517
heart_right_ventricle_2_1,18.584597,0.000000,0.000000,0.000000,0.000000,41.815343,2.323075,0.000000,0.000000,0.000000,2.323075,0.0,0.0,0.000000,0.000000,22.069209,1.161537
gm12878_3_1,17.413848,3.265096,0.000000,3.265096,5.441827,20.678944,0.000000,1.088365,1.088365,1.088365,4.353462,0.0,0.0,1.088365,0.000000,28.297503,3.265096
heart_right_ventricle_3_1,16.811502,0.000000,1.400959,0.000000,0.000000,37.825880,1.400959,0.000000,2.801917,0.000000,2.801917,0.0,0.0,0.000000,0.000000,11.207668,0.000000


In [27]:
# temp.loc[temp['ENSG00000171595[1,1,1]'] > 0]

In [28]:
sg = swan.read('swan/swan.p')
obs_col = 'sample'

Read in graph from swan/swan.p


In [29]:
filt_ab = 'cerberus_filtered_abundance.tsv'
t_df = pd.read_csv(filt_ab, sep='\t')

t_df = t_df[['annot_gene_name', 'annot_transcript_id', 'annot_gene_id']]
t_df.rename({'annot_gene_name': 'gname',
             'annot_gene_id': 'gid',
             'annot_transcript_id': 'tid'}, 
             axis=1, 
             inplace=True)

In [30]:
tpm_df = swan.calc_tpm(sg.adata, obs_col=obs_col, how='max').sparse.to_dense().transpose()
tpm_df.reset_index(inplace=True)
tpm_df.rename({'index':'tid'}, axis=1, inplace=True)

# melt to have one entry per tid / sample combination
def melt_transcript_sample(df, t_df, obs_col, col_name):
    df = df.merge(t_df[['tid', 'gname', 'gid']], how='inner', on='tid')
    df.set_index(['tid', 'gname', 'gid'], inplace=True)
    df = df.melt(ignore_index=False, value_name=col_name, var_name=obs_col)
    df = df.dropna(subset=[col_name])
    df.reset_index(inplace=True)
    return df 
tpm_df = melt_transcript_sample(tpm_df, t_df, obs_col, col_name='tpm')    


> /Users/fairliereese/miniconda3/lib/python3.7/site-packages/swan_vis/utils.py(493)calc_tpm()
    491                 df.reset_index(inplace=True)
    492                 if how == 'mean':
--> 493                         df = df.groupby(obs_col).mean()
    494                 elif how == 'max':
    495                         df = df.groupby(obs_col).max()



ipdb>  c


In [31]:
tpm_df.head()

,tid,gname,gid,sample,tpm
0,"ENSG00000227232[1,2,1]",WASH7P,ENSG00000227232.5,caco2,2.004524
1,"ENSG00000227232[1,3,1]",WASH7P,ENSG00000227232.5,caco2,2.597711
2,"ENSG00000227232[1,4,1]",WASH7P,ENSG00000227232.5,caco2,0.865904
3,"ENSG00000227232[1,5,1]",WASH7P,ENSG00000227232.5,caco2,1.002262
4,"ENSG00000227232[2,2,1]",WASH7P,ENSG00000227232.5,caco2,0.000000


In [32]:
# so, this gene should be showing up...
tpm_df.loc[(tpm_df.tid == 'ENSG00000149932[1,1,1]')&(tpm_df.tpm > 0)]

,tid,gname,gid,sample,tpm
173214,"ENSG00000149932[1,1,1]",TMEM219,ENSG00000149932.16,caco2,15.033932
406133,"ENSG00000149932[1,1,1]",TMEM219,ENSG00000149932.16,brain,91.544647
639052,"ENSG00000149932[1,1,1]",TMEM219,ENSG00000149932.16,mcf10a,19.726521
871971,"ENSG00000149932[1,1,1]",TMEM219,ENSG00000149932.16,a673,11.676455
1104890,"ENSG00000149932[1,1,1]",TMEM219,ENSG00000149932.16,lung,59.093750
1337809,"ENSG00000149932[1,1,1]",TMEM219,ENSG00000149932.16,hl60_m0,30.234989
1570728,"ENSG00000149932[1,1,1]",TMEM219,ENSG00000149932.16,colon,52.375900
1803647,"ENSG00000149932[1,1,1]",TMEM219,ENSG00000149932.16,hmec,29.803123
2036566,"ENSG00000149932[1,1,1]",TMEM219,ENSG00000149932.16,k562,18.756891
2269485,"ENSG00000149932[1,1,1]",TMEM219,ENSG00000149932.16,imr90,13.064860


In [21]:
# 0.665529*2

In [26]:
filt_ab = 'cerberus_filtered_abundance.tsv'
t_df = pd.read_csv(filt_ab, sep='\t')

In [27]:
t_df.head()

,gene_ID,transcript_ID,annot_gene_id,annot_transcript_id,annot_gene_name,annot_transcript_name,n_exons,length,gene_novelty,transcript_novelty,...,adrenal_gland_1_1,h1_de_1_1,heart_right_ventricle_3_1,right_cardiac_atrium_1_1,ovary_3_1,huvec_1_2,right_cardiac_atrium_4_1,wtc11_1_2,kidney_1_1,heart_left_ventricle_2_1
0,2,1150534,ENSG00000227232.5,"ENSG00000227232[1,2,1]",WASH7P,"WASH7P[1,2,1]",9.0,2065.0,Known,NIC,...,0,0,1,0,0,2,0,0,0,0
1,2,207076,ENSG00000227232.5,"ENSG00000227232[1,3,1]",WASH7P,"WASH7P[1,3,1]",10.0,1828.0,Known,NIC,...,0,0,2,0,0,1,0,1,2,1
2,2,207057,ENSG00000227232.5,"ENSG00000227232[1,4,1]",WASH7P,"WASH7P[1,4,1]",10.0,2164.0,Known,NIC,...,0,0,0,1,0,0,2,1,0,0
3,2,207067,ENSG00000227232.5,"ENSG00000227232[1,5,1]",WASH7P,"WASH7P[1,5,1]",11.0,1922.0,Known,NIC,...,0,0,2,2,1,1,2,1,2,0
4,2,1150534,ENSG00000227232.5,"ENSG00000227232[2,2,1]",WASH7P,"WASH7P[2,2,1]",9.0,2065.0,Known,NIC,...,0,0,0,0,0,0,0,0,0,0


In [30]:
c = [col for col in t_df.columns if 'hl60_m2_24hr' in col]
print(c)
t_df = t_df[['annot_transcript_id']+c]

['hl60_m2_24hr_1_1', 'hl60_m2_24hr_1_2']


In [32]:
t_df.loc[t_df.annot_transcript_id == 'ENSG00000269471[1,1,1]']

,annot_transcript_id,hl60_m2_24hr_1_1,hl60_m2_24hr_1_2
204788,"ENSG00000269471[1,1,1]",0,1
